In [47]:
%pip install tensorflow




In [48]:
import pandas as pd
import numpy as np
import pickle
from tensorflow.keras.models import load_model
import tensorflow as tf

In [49]:
#load trained modle
model =load_model("model.h5")

with open("scaler.pkl", "rb") as f:
    scaler = pickle.load(f)

with open("label_encoder_gender.pkl","rb") as f:
    gender_encoder=pickle.load(f)

with open("onehot_encoder_geo.pkl","rb") as f:
    geo_onehot_encoder=pickle.load(f)

In [50]:
input_data ={
    "CreditScore":600,
    "Geography":"France",
    "Gender":"Male",
    "Age":42,
    "Tenure":3,
    "Balance":100000,
    "NumOfProducts":2,
    "HasCrCard":1,
    "IsActiveMember":1,
    "EstimatedSalary":50000
}

In [51]:
geo_encoded = geo_onehot_encoder.transform(np.array([[input_data["Geography"]]])).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=geo_onehot_encoder.get_feature_names_out(["Geography"]))

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [52]:
#chagning the Geography
input_data_df = pd.DataFrame([input_data])
input_data = pd.concat([input_data_df.reset_index(drop=True), geo_encoded_df], axis=1)


In [53]:
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,Male,42,3,100000,2,1,1,50000,1.0,0.0,0.0


In [54]:
#encoder categorical variable
input_data["Gender"] = gender_encoder.fit_transform(input_data["Gender"])

In [55]:
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,0,42,3,100000,2,1,1,50000,1.0,0.0,0.0


In [56]:
input_data.drop(columns=["Geography"],inplace=True)

In [57]:
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,0,42,3,100000,2,1,1,50000,1.0,0.0,0.0


In [58]:
input_scaled = scaler.transform(input_data)
input_scaled

array([[-0.53598516, -1.09499335,  0.29493847, -0.69539349,  0.38262839,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [61]:
prediction=model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


array([[0.11995398]], dtype=float32)

In [63]:
prediction_proba = prediction[0][0]
prediction_proba

np.float32(0.119953975)

In [64]:
if prediction_proba > 0.5:
    print("Customer will leave the bank")
else:
    print("Customer will not leave the bank")

Customer will not leave the bank
